# Module 2

## 2.0 Загрузка данных

### Импорт библиотек для загрузки данных

In [1]:
# Импорт модуля pandas
import pandas as pd
import numpy as np
# Игнорирование предупреждений
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузка данных цели 1
target = pd.read_csv (r'target.csv')
target = target.drop('Unnamed: 0', axis=1)
print (target) # Просмотр таблицы
# Загрузка данных цели 2
SOP = pd.read_csv (r'SOP.csv')
SOP = SOP.drop('Unnamed: 0', axis=1)
print (SOP) # Просмотр таблицы

    Начало дефекта  Длина дефекта  Высота дефекта Тип дефекта Сторона
0               51           42.0               7           L       A
1              200           40.0               6           L       B
2              278           39.0               4           L       A
3              365           43.0               6           L       B
4              460           27.0               5           L       A
5              845           36.0               4           L       B
6              959           62.0               6           L       A
7              178           37.0               6           L       B
8              258           38.0               4           L       A
9              337           53.0               5           L       B
10             822           37.0               5           L       B
11             957           48.0               5           L       A
12              26           42.0               5           L       A
13             172  

## 2.1 Разбиение данных на обучающую и тестирующую выборки

В качестве тренировочной последовательности будет использоваться выборка SOP, полученная в результате сканирования системой ПУЗК стандартного образца предприятия (СОП).

Разбиение набора данных на обучающую и тестирующую выборки будем осуществлять через скейлирование из библиотеки sklearn, потому что это наиболее быстрый и оптимальный способ. Выборку будем делать предварительно и в будущем соотношение может поменяться. Пока оставляем 66% обучающей и 33% тестирующей с рандомным шагом 42.

In [3]:
# Масштабирование таблицы
X = SOP.drop(['intro'], axis=1)
y = SOP['intro']

# Загружаем библиотеку для разделения на выборки
from sklearn.model_selection import train_test_split

# Разделение в соотношении 66/33 с шагом 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

## 2.2 Аргументированный выбор модели классификации

Теперь надо понять, какую модель классификации выбрать и обучить на тестовых данных. Для этого возьмём 3 метода классификации: RF, KNN, GBT, посмотрим какой метод справился лучше всех.

### Random Forest

Алгоритм строит ансамбль случайных деревьев, каждое из которых обучается на выборке, полученной из исходной с помощью процедуры изъятия с возвращением.

In [4]:
# Подсчёт времени выполнения задачи
import time
start_time = time.time()

# Импортируем библиотеку для обучения
from sklearn import ensemble
import numpy as np

#Обучаем модель
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train, y_train)

# Оценка ошибок
err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))

# Вывод времени выполнения
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the 

--- 0.4574458599090576 seconds ---


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [5]:
#Вывод процента ошибок
print(err_train, err_test)

0.0 0.036893203883495145


Точность 99.9% и 99.9%. Время выполнения 0.3 ms.

### Классификация K-Neighbors

Для нового объекта алгоритм ищет в обучающей выборке k наиболее близких объекта и относит новый объект к тому классу, которому принадлежит большинство из них.

In [6]:
# Подсчёт времени выполнения задачи
import time
start_time = time.time()

# Импортируем библиотеку для обучения
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

#Обучаем модель
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(X_train, y_train)
y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

# Оценка ошибок
err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

# Вывод времени выполнения
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:908: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._y = np.empty(y.shape, dtype=np.int)


--- 0.21420812606811523 seconds ---


In [7]:
#Вывод процента ошибок
print(err_train, err_test)

0.03590234561991384 0.040776699029126215


Точность 99.9% и 99.9%. Время выполнения 0.22 ms.

### Классификация Gradient Boosting

Строит ансамбль деревьев решений. На каждой итерации строится новый классификатор, аппроксимирующий значение градиента функции потерь.

In [8]:
# Подсчёт времени выполнения задачи
import time
start_time = time.time()

# Импортируем библиотеку для обучения
from sklearn import ensemble
import numpy as np

#Обучаем модель
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(X_train, y_train)

# Оценка ошибок
err_train = np.mean(y_train != gbt.predict(X_train))
err_test  = np.mean(y_test  != gbt.predict(X_test))

# Вывод времени выполнения
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1348: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.object)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1570: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_mask = np.ones((n_samples, ), dtype=np.bool)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramDa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramDa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramDa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramDa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1206: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
C:\ProgramDa

--- 5.857126235961914 seconds ---


In [9]:
#Вывод процента ошибок
print(err_train, err_test)

0.0 0.035922330097087375


Точность 100% и 99.9%. Время выполнения 5.5 sec.

Все методы классификации показали себя хорошо, точность более 80% - отличный результат. Если точность у всех моделей ± одинаковая, то выбираем модель, которая не задействует значительные ресурсы вычислительной машины (работает быстрее всего)

**Вывод: Метот Random Forest**

### ROC-кривая

In [ ]:
# Импорт библиотек
import sklearn.linear_model as sk
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#Создание и ввод переменных
logreg = sk.LogisticRegressionCV()
logreg.fit(X_train, y_train)
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])

#Построение графика
plt.figure()
plt.plot(fpr, tpr, label='Логистическая регрессия (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Отрицательный')
plt.ylabel('Позитивный')
plt.title('ROC-кривая')
plt.legend(loc="lower right")
plt.show()

## Отчёт

In [ ]:
#Сохранение HTML
!jupyter nbconvert С2_M2.ipynb --to html